In [1]:
from config import *
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
import torch.nn.functional as F
import os
from data.data_loader import *
from model import *
import torch.optim as optim

batchSize = 2

print("Loading the dataset")
Source_x, Source_y, Target_x = prepare_Data(data_dir, True)
Gallery_x, Query_x = prepare_Data(data_dir, False)
label_Similarity = csr_matrix(scipy.io.loadmat("data/cifar10/cifar10_Similarity.mat")['label_Similarity']).todense()
print("Data loading finished")

source = torch.utils.data.DataLoader([(Source_x[i], Source_y[i]) for i in range(len(Source_x))],batch_size=batchSize)
target = torch.utils.data.DataLoader(Target_x,batch_size=batchSize)

Loading the dataset
--------------------Loading dataset--------------------
Source:  (5000, 32, 32, 3) (5000,)
Target:  (54000, 32, 32, 3)
--------------------Loading dataset--------------------
Gallery:  (54000, 32, 32, 3) (54000,)
Query:  (1000, 32, 32, 3) (1000,)
Data loading finished


In [2]:
vggModel=models.vgg16_bn(pretrained=True)
# vggModel.children

In [3]:
net1, net2 = [], []
for i in vggModel.children():
    for r, i in enumerate(i.children()):
        if r <=23: net1.append(i)
        elif r<= 32: net2.append(i)
    break
net1, net2  = nn.Sequential(*net1), nn.Sequential(*net2)

In [4]:
def intranorm(features,n_book):
    x = features.split(n_book,1)
    
    for b in range(n_book):
        if b==0: dummy = F.normalize(x[b],1)
        else:
            dummy = torch.cat((dummy,F.normalize(x[b],1)),1)
    return dummy

In [5]:
def shape_(inp):
     for i in inp:
            print(f"shape is: {i.shape}")

In [163]:
class softassignment_(nn.Module):
    def __init__(self,len_code, n_book, intn_word):
        super(softassignment_,self).__init__()
        self.Z = nn.Linear(len_code * n_book,intn_word, bias=False)

    def forward(self,Z,features,n_book,alpha,):
        z_ = Z.split(n_book,1)
        x_ = features.split(n_book,1)

        for i in range(n_book):
            size_z = z_[i].shape[0] # number of codewords
            size_x = x_[i].shape[0] # batch size
            xx = x_[i].unsqueeze(-1)
            xx = xx.repeat(1,1,size_z)
            zz = z_[i].unsqueeze(-1)
            zz = zz.repeat(1,1,size_x).T

            diff = 1 - torch.sum(torch.mul(xx,zz), 1) # 32,16
            softmax_diff = F.softmax(diff*(-alpha),1) #32,16
            soft_des_temp = torch.matmul(softmax_diff,z_[i]) # 32,12
            if i == 0: descriptor = soft_des_temp
            else: descriptor = torch.cat((descriptor,soft_des_temp),1)

        return intranorm(descriptor,n_book) # 32,144

In [164]:
class classifier_(nn.Module):
    def __init__(self,n_CLASSES, len_code, n_book,):
        super(classifier_,self).__init__()
        self.prototypes = nn.Linear(len_code * n_book, n_CLASSES, bias=False) #[]
#         for i in range(n_book):
#             self.prototypes.append(nn.init.xavier_normal_(torch.empty(n_CLASSES, len_code * n_book)))
        
    def forward(self, x,c):
        x_ = x.split(n_book,1)
        c_ = c.split(n_book,0)
#         shape_(x_[0])
#         shape_(c_[0])
        for i in range(n_book):
            sub_res = torch.matmul(x_[i], c_[i]).unsqueeze(-1)
            if i == 0: res = sub_res
            else: res = torch.cat((res,sub_res),2)
        
        return torch.sum(res, 2)

In [165]:
class features_(nn.Module):
    def __init__(self, net1, net2):
        super(features_,self).__init__()
        
        self.net1 = net1
        self.net2 = net2
        self.gavgp = nn.AdaptiveAvgPool2d(1)
        self.linear = nn.Linear(768, len_code*n_book)
        
    def forward(self,x):
        x = self.net1(x) # shape: torch.Size([32, 3, 32,32])>torch.Size([32, 3, 4, 4])
        x_branch = self.gavgp(x)
        x = self.net2(x) # shape: torch.Size([32, 3, 4, 4])> torch.Size([32, 3, 4, 4])
        x = self.gavgp(x)
        
        x = torch.cat((x,x_branch),1)
        
        return self.linear(x.view(-1,768))
    
# x = torch.randn(32, 3, 32, 32, device='cpu')
# model = features_(net1, net2)
# out = model(x)# model
# out.shape, out.split(12,1)[0].shape, model.Z.shape, model.prototypes.shape

In [174]:
#models
model = features_(net1, net2).to(device)
classifier = classifier_(n_CLASSES, len_code, n_book).to(device)
softassignment = softassignment_(len_code, n_book, intn_word).to(device)

# optimizer
class_optim = optim.Adam(classifier.parameters(),lr=0.0002,betas=(0.5,0.999))
model_optim = optim.Adam(model.parameters(),lr=0.0002,betas=(0.5,0.999))
soft_optim = optim.Adam(softassignment.parameters(),lr=0.0002,betas=(0.5,0.999))

prototypes = intranorm(classifier.prototypes.state_dict()['weight'], n_book).to(device)
Z = intranorm(softassignment.Z.state_dict()['weight'], n_book).to(device)

In [175]:
target_ = iter(target)
for epoch in range(1):
    for batch in source:
        x, y = batch
        x = torch.tensor(data_augmentation(x)).to(device)
        try: 
            xu = next(target_)
            xu = torch.tensor(data_augmentation(xu)).to(device)
        except:
            target_ = iter(target)
            xu = next(target_)
            xu = torch.tensor(data_augmentation(xu)).to(device)
        shape_([x,y,xu])
        
        
        y_ = y.to(device)
        y=torch.eye(numClasses)[y]
        ymat=torch.matmul(y,y.T)
        ymat/=torch.sum(ymat,axis=1,keepdims=True)
        ymat = ymat.to(device)
#         y = y.to(device)
        
        
        features = intranorm(model(x.permute(0,3,1,2)), n_book)
        quanta = softassignment(Z,features,n_book,alpha)
        logits = classifier(features, prototypes.T)
        
        hash_loss = NPQLoss(ymat,features, quanta,n_book)
        print(hash_loss)
        cls_loss = torch.nn.functional.cross_entropy(logits,y_)
        entropy_loss = SMELoss(features * beta, prototypes * beta, n_book)
        final_loss = hash_loss + lam_1*entropy_loss + lam_2*cls_loss 
        
        break
        optimizer.zero_grad()
        final_loss.backward(retain_graph=True)
        optimizer.step()
        
        break

shape is: torch.Size([2, 32, 32, 3])
shape is: torch.Size([2])
shape is: torch.Size([2, 32, 32, 3])
tensor(0.3484, device='cuda:0', grad_fn=<AddBackward0>)


In [6]:
def softassignment(Z,features,n_book,alpha,):
        z_ = Z.split(n_book,1)
        x_ = features.split(n_book,1)
        
        for i in range(n_book):
            size_z = z_[i].shape[0] # number of codewords
            size_x = x_[i].shape[0] # batch size
            xx = x_[i].unsqueeze(-1)
            xx = xx.repeat(1,1,size_z)
            zz = z_[i].unsqueeze(-1)
            zz = zz.repeat(1,1,size_x).T
            
            diff = 1 - torch.sum(torch.mul(xx,zz), 1) # 32,16
            softmax_diff = F.softmax(diff*(-alpha),1) #32,16
            soft_des_temp = torch.matmul(softmax_diff,z_[i]) # 32,12
            if i == 0: descriptor = soft_des_temp
            else: descriptor = torch.cat((descriptor,soft_des_temp),1)
            
        return intranorm(descriptor,n_book) # 32,144